In [1]:
import pygame, sys, random
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
import math
import matplotlib.pyplot as plt
import pylab
from fractions import Fraction

num_episodes = 10

obstacleRadius = 10
agentRadius = 10

# get size of state and action from environment

boundaryPos = [100, 100]
boundaryLength = [70,70]
boundaryRadius = 40
dispSize = [1280, 960]
initPosAgentStandard = [dispSize[0] - 100, 100]#dispSize[1]]
initPosAgent = initPosAgentStandard#[boundaryPos[0] + boundaryLength[0] / 2, boundaryPos[1] + boundaryLength[1] / 2]
goalPos = [100, 900]
goalAngle = 0#random.randrange(0, 360) * math.pi / 180

obstacleRandomRange = 1000

moveObstacles = True
action_size = 8
obsNumber = 100
state_size = 2
# state_size = obsNumber + 1

Using TensorFlow backend.


In [2]:
# A2C(Advantage Actor-Critic) agent
class A2CAgent:
    def __init__(self, state_size, action_size):
        self.load_model1 = True
        self.load_model2 = False
        
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        self.value_size = 1

        # These are hyper parameters for the Policy Gradient
        self.discount_factor = 0.99
        self.actor_lr = 0.00002
        self.critic_lr = 0.00005

        # create model for policy network
        self.actor = self.build_actor()
        self.critic = self.build_critic()

        if self.load_model1:
            self.actor.load_weights("./Practice004_DataSave/Actor_PEARL.h5")
            self.critic.load_weights("./Practice004_DataSave/Critic_PEARL.h5")

#             print self.actor.weights
#             print self.critic.weights
    # approximate policy and value using Neural Network
    # actor: state is input and probability of each action is output of model
    def build_actor(self):
        actor = Sequential()
        actor.add(Dense(1, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        actor.add(Dense(self.action_size, activation='softmax', kernel_initializer='glorot_normal'))
        actor.summary()
        # See note regarding crossentropy in cartpole_reinforce.py
        actor.compile(loss='categorical_crossentropy', optimizer=Adam(lr=self.actor_lr))
        return actor

    # critic: state is input and value of state is output of model
    def build_critic(self):
        critic = Sequential()
        critic.add(Dense(1, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        critic.add(Dense(self.value_size, activation='linear', kernel_initializer='glorot_normal'))
        critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return critic

    # using the output of policy network, pick action stochastically
    def get_action(self, state):
        policy = self.actor.predict(state, batch_size=1).flatten()
#         print policy
        return policy
#         return np.random.choice(self.action_size, 1, p=policy)[0]

    # update policy network every episode
    def train_model(self, state, action, reward, next_state, done):
        target = np.zeros((1, self.value_size))
        advantages = np.zeros((1, self.action_size))

        value = self.critic.predict(state)[0]
        next_value = self.critic.predict(next_state)[0]

        if done:
            advantages[0][action] = reward - value
            target[0][0] = reward
        else:
            advantages[0][action] = reward + self.discount_factor * (next_value) - value
            target[0][0] = reward + self.discount_factor * next_value

        self.actor.fit(state, advantages, epochs=1, verbose=0)
        self.critic.fit(state, target, epochs=1, verbose=0)

In [3]:
def stateGenerator(intenPref, distPref):
    returnSum = [intenPref, distPref]
    returnSum = np.reshape(returnSum, [1, state_size])
#     print returnSum
    return returnSum

In [4]:
def takeAction(action):
    xAction = 0
    yAction = 0
    if action == 0:
        xAction = 1
    elif action == 1:
        xAction = 1
        yAction = 1
    elif action == 2:
        xAction = 1
        yAction = -1            
    elif action == 3:
        xAction = -1
        yAction = 1
    elif action == 4:
        xAction = -1
    elif action == 5:
        xAction = -1
        yAction = -1
    elif action == 6:
        yAction = -1
    elif action == 7:
        yAction = 1
#     elif action == 8:
#         xAction = 0
#         yAction = 0
        
    return [xAction, yAction]

In [5]:
def makeFeature(agtPos, goalPos, obsPos, const):
    feature1 = (agtPos[0] - goalPos[0])**2 + (agtPos[1] - goalPos[1])**2
    feature2 = 1 / ((agtPos[0] - obsPos[0])**2 + (agtPos[1] - obsPos[1])**2 + const)
    return [feature1, feature2]

In [6]:
def rangeFinder(allObsPos, rangeCenter):
    allObsAgtDistance = [0 for _ in range(obsNumber)]
    for i in range(0, obsNumber):
        allObsAgtDistance[i] = math.sqrt((allObsPos[i][0] - rangeCenter[0])**2 + (allObsPos[i][1] - rangeCenter[1])**2)
    index = np.argmin(allObsAgtDistance)
    return index

In [ ]:
pygame.init()
screen = pygame.display.set_mode(dispSize)
screen.fill([200, 200, 200])

# make A2C agent
agent = A2CAgent(state_size, action_size)

rList, episodes = [], []

# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
for i in range(0,obsNumber):
    obstaclePos[i][0] = int(initPosAgent[0] - (obstacleRadius + agentRadius + random.randrange(0, dispSize[0])))
    obstaclePos[i][1] = int(initPosAgent[1] + (obstacleRadius + agentRadius + random.randrange(0, dispSize[1])))

for e in range(num_episodes):
    # Initialize
    done = False
    score = 0
    x = initPosAgent[0]
    y = initPosAgent[1]
    print("Episode ", e, "Starts!")
    xMove = 0
    yMove = 0
    while not done:
        tmpAction = []
        idx = rangeFinder(obstaclePos, [x, y])
        FeatureVec1 = (x - goalPos[0])**2 + (y - goalPos[1])**2
        FeatureVec2 = 1 / (0.2 + (x - obstaclePos[idx][0])**2 + (y - obstaclePos[idx][1])**2)
        state = stateGenerator(FeatureVec1, FeatureVec2)
        policyArr = agent.get_action(state)

        action = np.random.choice(action_size, 1, p = policyArr)[0]

        [xMove, yMove] = takeAction(action)

        x = x + xMove
        y = y + yMove
        
     
        wallFlag = 0
        collisionFlag = 0
        pygame.draw.circle(screen, [100, 100, 255], [x,y], 10, 0)

        initPosAgent = [x,y]
   
        if math.sqrt((x -  goalPos[0])**2 + (y - goalPos[1])**2) <= agentRadius:
            print("Goal Reached!")
            collisionFlag = 1
            done = True
        for i in range(0,obsNumber):
            if moveObstacles:
                obstaclePos[i][0] = obstaclePos[i][0] + random.randrange(-1,2)
                obstaclePos[i][1] = obstaclePos[i][1] + random.randrange(-1,2)
                
            pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], obstacleRadius, 0)
            if math.sqrt((x - obstaclePos[i][0])**2 + (y - obstaclePos[i][1])**2) < 19:
                print("Collision!")
                collisionFlag = -1
                done = True
        
        pygame.draw.circle(screen, [255,100,100], initPosAgent, boundaryRadius, 2)

        if done:
            # every episode, plot the play time
            initPosAgent = initPosAgentStandard
            obstaclePos = [[0, 0] for _ in range(obsNumber)]
            for i in range(0,obsNumber):
                obstaclePos[i][0] = int(initPosAgent[0] - (obstacleRadius + agentRadius + random.randrange(0, dispSize[0])))
                obstaclePos[i][1] = int(initPosAgent[1] + (obstacleRadius + agentRadius + random.randrange(0, dispSize[1])))

            episodes.append(e)
            
        pygame.draw.circle(screen, [100,255,100], goalPos, 10, 2)
        pygame.display.flip()
        screen.fill([220,220,220])
    print score


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1)                 3         
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 16        
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 1)                 3         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 2         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________
('Episode ', 0, 'Starts!')


In [14]:
print("Percent of successful episodes: " + str(100.0 * sum(rList)/num_episodes) + "%")

# plt.bar(range(len(rList)), rList, color = "Blue", width = 0.00001)
# plt.show()

Percent of successful episodes: 40.0%
